# BM25 Retriever
In this guide, we define a bm25 retriever that search documents using bm25 method.

This notebook is very similar to the RouterQueryEngine notebook.

## Setup

In [1]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

In [2]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "sk-..."
openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().handlers = []
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import (
    ListIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    SimpleKeywordTableIndex,
    VectorStoreIndex,
)
from llama_index.retrievers import BM25Retriever
from llama_index.indices.vector_store.retrievers.retriever import VectorIndexRetriever
from llama_index.llms import OpenAI

NumExpr defaulting to 8 threads.


## Load Data

We first show how to convert a Document into a set of Nodes, and insert into a DocumentStore.

In [4]:
# load documents
documents = SimpleDirectoryReader("../data/paul_graham").load_data()

In [5]:
# initialize service context (set chunk size)
llm = OpenAI(model="gpt-4")
service_context = ServiceContext.from_defaults(chunk_size=1024, llm=llm)
nodes = service_context.node_parser.get_nodes_from_documents(documents)

In [6]:
# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [7]:
index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context,
    service_context=service_context,
)

## BM25 Retriever

We will search document with bm25 retriever.

In [ ]:
# !pip install rank_bm25

In [8]:
from llama_index.utils import globals_helper

retriever = BM25Retriever(index.docstore, globals_helper.tokenizer)

In [9]:
from llama_index.response.notebook_utils import display_source_node

# will retrieve all context from the author's life
nodes = retriever.retrieve(
    "Can you give me all the context regarding the author's life?"
)
for node in nodes:
    display_source_node(node)

**Node ID:** e88fa565-97c4-417b-bd38-86f55be57f7e<br>**Similarity:** 7.322087057741413<br>**Text:** This name didn't last long before it was replaced by "software as a service," but it was current ...<br>

**Node ID:** 51903d9f-367f-4046-8f88-d34b3d1a3edc<br>**Similarity:** 7.121406119821665<br>**Text:** There, right on the wall, was something you could make that would last.Paintings didn't become ob...<br>

In [10]:
nodes = retriever.retrieve("What did Paul Graham do after RISD?")
for node in nodes:
    display_source_node(node)

**Node ID:** 952e4215-330b-4c67-9177-20d30168cda2<br>**Similarity:** 6.84581665569314<br>**Text:** That seemed unnatural to me, and on this point the rest of the world is coming around to my way o...<br>

**Node ID:** c9326cd9-4ab8-48df-8e7c-a642f9a4935a<br>**Similarity:** 6.259657209129812<br>**Text:** So I decided to take a shot at it.It took 4 years, from March 26, 2015 to October 12, 2019.It was...<br>

## Router Retriever with bm25 method

Now we will combine bm25 retriever with vector index retriever.

In [11]:
from llama_index.tools import RetrieverTool

vector_retriever = VectorIndexRetriever(index)
bm25_retriever = BM25Retriever(index.docstore, globals_helper.tokenizer)

retriever_tools = [
    RetrieverTool.from_defaults(
        retriever=vector_retriever,
        description="Useful in most cases",
    ),
    RetrieverTool.from_defaults(
        retriever=bm25_retriever,
        description="Useful if searching about specific information",
    ),
]

In [12]:
from llama_index.selectors.pydantic_selectors import PydanticMultiSelector
from llama_index.retrievers import RouterRetriever

retriever = RouterRetriever.from_defaults(
    retriever_tools=retriever_tools,
    service_context=service_context,
    select_multi=True,
)

In [13]:
# will retrieve all context from the author's life
nodes = retriever.retrieve(
    "Can you give me all the context regarding the author's life?"
)
for node in nodes:
    display_source_node(node)

Selecting retriever 1: This choice is relevant because it involves searching for specific information, which in this case would be details about the author's life..


**Node ID:** e88fa565-97c4-417b-bd38-86f55be57f7e<br>**Similarity:** 7.322087057741413<br>**Text:** This name didn't last long before it was replaced by "software as a service," but it was current ...<br>

**Node ID:** 51903d9f-367f-4046-8f88-d34b3d1a3edc<br>**Similarity:** 7.121406119821665<br>**Text:** There, right on the wall, was something you could make that would last.Paintings didn't become ob...<br>

## Advanced - Hybrid Retriever + Re-Ranking

Here we extend the base retriever class and create a custom retriever that always uses the vector retriever and BM25 retreiver.

Then, nodes can be re-ranked and filtered. This let's us keep intermediate top-k values large and letting the re-ranking filter out un-needed nodes.

To best demonstrate this, we will use a larger set of source documents -- Chapter 3 from the 2022 IPCC Climate Report.

### Setup data

In [14]:
!curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20.7M  100 20.7M    0     0  15.4M      0  0:00:01  0:00:01 --:--:-- 15.5M


In [ ]:
# !pip install pypdf

In [15]:
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    StorageContext,
    SimpleDirectoryReader,
)
from llama_index.llms import OpenAI

# load documents
documents = SimpleDirectoryReader(
    input_files=["IPCC_AR6_WGII_Chapter03.pdf"]
).load_data()

# initialize service context (set chunk size)
# -- here, we set a smaller chunk size, to allow for more effective re-ranking
llm = OpenAI(model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(chunk_size=256, llm=llm)
nodes = service_context.node_parser.get_nodes_from_documents(documents)

# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [16]:
index = VectorStoreIndex(
    nodes, storage_context=storage_context, service_context=service_context
)

In [18]:
from llama_index.retrievers import BM25Retriever
from llama_index.utils import globals_helper

# retireve the top 10 most similar nodes using embeddings
vector_retriever = index.as_retriever(similarity_top_k=10)

# retireve the top 10 most similar nodes using bm25
bm25_retriever = BM25Retriever(index.docstore, globals_helper.tokenizer, similarity_top_k=10)

### Custom Retriever Implementation

In [19]:
from llama_index.retrievers import BaseRetriever


class HybridRetriever(BaseRetriever):
    def __init__(self, vector_retriever, bm25_retriever):
        self.vector_retriever = vector_retriever
        self.bm25_retriever = bm25_retriever

    def _retrieve(self, query, **kwargs):
        bm25_nodes = self.bm25_retriever.retrieve(query, **kwargs)
        vector_nodes = self.vector_retriever.retrieve(query, **kwargs)

        # combine the two lists of nodes
        all_nodes = []
        node_ids = set()
        for n in bm25_nodes + vector_nodes:
            if n.node.node_id not in node_ids:
                all_nodes.append(n)
                node_ids.add(n.node.node_id)
        return all_nodes

In [20]:
index.as_retriever(similarity_top_k=5)

hybrid_retriever = HybridRetriever(vector_retriever, bm25_retriever)

### Re-Ranker Setup

In [21]:
from llama_index.indices.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(
    top_n=4, model="cross-encoder/ms-marco-MiniLM-L-6-v2"
)

/Users/jasony/Projects/llama_index/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

Use pytorch device: cpu


### Retrieve

In [22]:
from llama_index import QueryBundle

nodes = hybrid_retriever.retrieve("What is the impact of climate change on the ocean?")
reranked_nodes = reranker.postprocess_nodes(
    nodes,
    query_bundle=QueryBundle("What is the impact of climate change on the ocean?"),
)

print("Initial retrieval: ", len(nodes), " nodes")
print("Re-ranked retrieval: ", len(reranked_nodes), " nodes")

Batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.98it/s]

Initial retrieval:  19  nodes
Re-ranked retrieval:  4  nodes


In [23]:
from llama_index.response.notebook_utils import display_source_node

for node in reranked_nodes:
    display_source_node(node)

**Node ID:** 543a0552-4ea5-4e82-af71-bb25aef524f1<br>**Similarity:** 6.131189346313477<br>**Text:** Observations: vulnerabilities and impacts
Anthropogenic climate change has exposed ocean and coas...<br>

**Node ID:** 02a57ee5-2753-4eb2-b9f2-ed636739658f<br>**Similarity:** 6.0154032707214355<br>**Text:** 3
469Oceans and Coastal Ecosystems and Their Services  Chapter 3
Frequently Asked Questions
FAQ 3...<br>

**Node ID:** 2cb736c0-4ca3-4d7f-931d-9f03218841f8<br>**Similarity:** 4.70263671875<br>**Text:** {Box 3.2, 3.2.2.1, 3.4.2.5, 3.4.2.10, 3.4.3.3, Cross-Chapter 
Box PALEO in Chapter 1}
Climate imp...<br>

**Node ID:** 6099b6cc-1244-4250-ae11-75c89978c899<br>**Similarity:** 3.768510341644287<br>**Text:** In both polar regions, climate-induced changes in ocean and sea ice conditions have 
expanded the...<br>

### Full Query Engine 

In [24]:
from llama_index.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(
    retriever=hybrid_retriever,
    node_postprocessors=[reranker],
    service_context=service_context,
)

response = query_engine.query("What is the impact of climate change on the ocean?")

Batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.24it/s]


In [25]:
from llama_index.response.notebook_utils import display_response

display_response(response)

**`Final Response:`** Climate change has greatly impacted life in the ocean and along its coasts. It has exposed ocean and coastal ecosystems to conditions that are unprecedented over millennia. Fundamental changes in the physical and chemical characteristics of the ocean are changing the timing, distribution, and abundance of oceanic and coastal organisms. These changes have been observed through multi-decadal observations, laboratory studies, and meta-analyses of published data. Additionally, climate change is degrading ocean health, altering stocks of marine resources, and threatening the sustenance provided to Indigenous Peoples, livelihoods of artisanal fisheries, and marine-based industries such as tourism, shipping, and transportation. The impacts of climate change on the ocean can influence human activities and employment by altering resource availability, spreading pathogens, flooding shorelines, and degrading ocean ecosystems.